In [2]:
# Import the needed packages
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import chart_studio.plotly as py
from plotly.subplots import make_subplots
import cufflinks as cf
import seaborn as sns # for sample datasets only!
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [5]:
# Import all the needed files for the analysis
campaign_table = pd.read_csv('Retail Data Viz/Dunnhumby Datasets/campaign_table.csv')
causal_data = pd.read_csv('Retail Data Viz/Dunnhumby Datasets/causal_data.csv')
coupon = pd.read_csv('Retail Data Viz/Dunnhumby Datasets/coupon.csv')
campaign_desc = pd.read_csv('Retail Data Viz/Dunnhumby Datasets/campaign_desc.csv')
product = pd.read_csv('Retail Data Viz/Dunnhumby Datasets/product.csv')
transaction_data = pd.read_csv('Retail Data Viz/Dunnhumby Datasets/transaction_data.csv')
hh_demographic = pd.read_csv('Retail Data Viz/Dunnhumby Datasets/hh_demographic.csv')
coupon_redempt = pd.read_csv('Retail Data Viz/Dunnhumby Datasets/coupon_redempt.csv')

Examine the transaction table

In [4]:
transaction_data.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [5]:
transaction_data.shape

(2595732, 12)

In [6]:
# Subset the data to make charts run faster
transaction_data = transaction_data[transaction_data['DAY'] < 100]

In [7]:
transaction_data.shape

(158305, 12)

### Data Transformations and Cleaning

In [55]:
# Create Daily aggregate data for product trends
transaction_data = transaction_data[['household_key','DAY','PRODUCT_ID', 'QUANTITY','SALES_VALUE']] \
            .merge(product[['PRODUCT_ID','COMMODITY_DESC']], on='PRODUCT_ID')

# Remove blanks and 'COUPON/MISC ITEMS', and '(CORP USE ONLY)' in the COMMODITY_DESC field as they won't be helpful in the analysis anyway
transaction_data = transaction_data[~transaction_data['COMMODITY_DESC'].isin(['',' ','COUPON/MISC ITEMS','(CORP USE ONLY)'])] 

# Do a daily summary with the following metrics: sales, quantity, number of households
daily_sales = transaction_data.groupby(['COMMODITY_DESC', 'DAY']) \
            .agg({'SALES_VALUE':'sum', 'QUANTITY':'sum', 'household_key':pd.Series.nunique}) \
            .rename(columns = {'household_key':'HOUSEHOLDS'}) \
            .reset_index()

daily_sales.head()

## Plotly Charts

### Chart 1: Bar Chart, Top 5 Commodities in terms of Sales

In [48]:
# Prepare dataframes
df_top5 = daily_sales[['COMMODITY_DESC','SALES_VALUE']]\
            .groupby(['COMMODITY_DESC']).sum().reset_index()
df_top5 = df_top5[df_top5['COMMODITY_DESC'] != ''].sort_values(by = 'SALES_VALUE', ascending=False)[:5] \
            .sort_values(by = 'SALES_VALUE')

In [49]:
# Bar Charts Using Plotly Express
fig = px.bar(x = df_top5.SALES_VALUE, 
             y = df_top5.COMMODITY_DESC,
             labels = {
                 'y' : 'Commodities',
                 'x' : 'Sales'
             },
             title = 'Top 5 Commodities by Sales',
             template = 'simple_white')
fig.show()

### Chart 2: Two Bar Charts, Top 5 Commodities in terms of Sales, and Top 5 in terms of Quantity

In [50]:
# Prepare dataframe for quantity
df_top5_quantity = daily_sales[['COMMODITY_DESC','QUANTITY']]\
            .groupby(['COMMODITY_DESC']).sum().reset_index()
df_top5_quantity = df_top5_quantity[df_top5_quantity['COMMODITY_DESC'] != ''] \
            .sort_values(by = 'QUANTITY', ascending=False)[:5]

df_top5 = df_top5.sort_values(by = 'SALES_VALUE', ascending=False)[:5]

In [51]:
# Add the 2 traces
trace1 = go.Bar(
            x = df_top5.COMMODITY_DESC,
            y = df_top5.SALES_VALUE,
            name = 'Sales $')

trace2 = go.Bar(
            x = df_top5_quantity.COMMODITY_DESC,
            y = df_top5_quantity.QUANTITY,
            name = 'Quantity')

# Set-up subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Sales $", "Quantity"))

fig.add_trace(
    trace1,
    row=1, col=1
)

fig.add_trace(
    trace2,
    row=1, col=2
)

fig.update_layout(height=600, width=800, template = 'simple_white', title_text="Top 5 Commodities in Sales and Quantity")

fig.show()

### Chart 3: Line Charts, Daily Sales for the Top 5 Commodities

In [52]:
# Prepare dataset
list_top5_sales = df_top5['COMMODITY_DESC']

daily_sales_top5 = daily_sales[daily_sales['COMMODITY_DESC'] \
                               .isin(list_top5_sales)] \
                                [['COMMODITY_DESC', 'SALES_VALUE', 'DAY']] \
                               .groupby(['COMMODITY_DESC','DAY']).sum().reset_index()

fig = px.line(daily_sales_top5, 
              x = daily_sales_top5.DAY, 
              y = daily_sales_top5.SALES_VALUE,
             color = daily_sales_top5.COMMODITY_DESC,
             title = 'Daily Sales for the Top 5 Commodities',
             template = 'simple_white')

fig.show()

### Chart 4: Box Plots for the Top 5 Commodities

In [53]:
fig = px.box(daily_sales_top5, 
              x = daily_sales_top5.COMMODITY_DESC, 
              y = daily_sales_top5.SALES_VALUE,
             color = daily_sales_top5.COMMODITY_DESC,
             title = 'Sales Distribution for the Top 5 Commodities',
             template = 'simple_white')
fig.show()

### Chart 5: Bubble Chart: Sales, Quantity, and Number of Households Buying  

In [60]:
# Do a summary with the following metrics: sales, quantity, number of households
sales_agg = transaction_data.groupby(['COMMODITY_DESC']) \
            .agg({'SALES_VALUE':'sum', 'QUANTITY':'sum', 'household_key':pd.Series.nunique}) \
            .rename(columns = {'household_key':'HOUSEHOLDS'}) \
            .reset_index()

In [84]:
fig = px.scatter(sales_agg, 
                 x='QUANTITY', 
                 y='SALES_VALUE',
                 size='HOUSEHOLDS', 
                 color='COMMODITY_DESC',
                 hover_name='COMMODITY_DESC',
                 size_max=60,
                 title = 'Sales, Volume, and Household Counts for Various Consumer Commodities',
                 template = 'simple_white')
fig.update_layout(showlegend=False)
fig.add_annotation(text='Sizes of the bubbles represent the number of households buying the product',
                  xref='paper', yref='paper',
                  x=-0.04, y=1.11, showarrow=False)
fig.show()